### Introducción
El objetivo final de este proyecto es predecir el sentimiento de un texto dado usando python, donde usamos NLTK, también conocido como Natural Language Processing Toolkit y de ser posible precisar el impacto causado por los comentarios positivos y negativos sobre un determinado comportamiento.

* Para este proyecto, importaremos un conjunto de librerías que se muestran a continuación
* Importaremos el conjunto de datos
* Realizaremos limpieza, formateo y visualización de los datos
* Construcción del modelo
* Predicción
* Medición de impacto 

In [2]:
# Importar librerías
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [32]:
# Importar datos
data = pd.read_csv('C:\\Users\\Joako\\Documents\\AnalisisDatos\\ReviewProject\\DatosOriginales\\reviews_filmaffinity.csv', delimiter='|', header=0)

In [33]:
data.shape

(8591, 6)

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8589 entries, 0 to 8590
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   film_name      8589 non-null   object 
 1   gender         8589 non-null   object 
 2   film_avg_rate  8589 non-null   float64
 3   review_rate    8589 non-null   float64
 4   review_title   8589 non-null   object 
 5   review_text    8589 non-null   object 
dtypes: float64(2), object(4)
memory usage: 469.7+ KB


In [45]:
data.head()

,film_name,gender,film_avg_rate,review_rate,review_title,review_text
0,Ocho apellidos vascos,Comedia,6.0,3.0,OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO,La mayor virtud de esta película es su existen...
1,Ocho apellidos vascos,Comedia,6.0,2.0,El perro verde,"No soy un experto cinéfilo, pero pocas veces m..."
2,Ocho apellidos vascos,Comedia,6.0,2.0,Si no eres de comer mierda... no te comas esta...,Si no eres un incondicional del humor estilo T...
3,Ocho apellidos vascos,Comedia,6.0,2.0,Aida: The movie,"No sé qué está pasando, si la gente se deja ll..."
4,Ocho apellidos vascos,Comedia,6.0,2.0,UN HOMBRE SOLO (Julio Iglesias 1987),"Pero cuando amanece,y me quedo solo,siento en ..."


In [36]:
# Encontrar nulos
data.isnull().sum()

film_name        0
gender           0
film_avg_rate    0
review_rate      2
review_title     0
review_text      0
dtype: int64

In [37]:
# Eliminar nulos
data=data.dropna()

In [38]:
# Comprobando datos de la columna y noto que los valores contienen una coma y además son objetos. 
film=data['film_avg_rate'].unique()
print(film)


['6,0' '6,8' '4,4' '6,7' '4,8' '5,3' '6,5' '6,4' '7,2' '4,2' '6,9' '3,7'
 '5,7' '5,5' '7,7' '6,1' '5,8' '5,2' '5,4' '4,6' '5,0' '7,1' '3,9' '4,7'
 '5,1' '6,2' '6,6' '7,0']


In [43]:
# Los convierto en float y remplazo ',' por '.'
data['film_avg_rate'] = data['film_avg_rate'].str.replace(',', '.').astype(float)
